<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/BB_seq_bn_P_3_seq_bn_C_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This creates one adapter for pre-training and another for classifcation.  

* The pre-training adapter is not trained during the classification, only the classifcation adapter.






# Setup

In [ ]:
from time import time
start_time = time()

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

In [ ]:
!pip install pynvml

In [ ]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [1]:
from datasets import load_dataset

pretrain_train_dataset, pretrain_eval_dataset = load_dataset("BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset", split=["test","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating dev split: 100%|██████████| 1980/1980 [00:00<00:00, 170409.20 examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9754
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [2]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/afm/.pyenv/versions/3.11.3/envs/adapt/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'head

In [11]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

pretraining_adapter_config = SeqBnConfig()
pretrained_adapter_name = "tapt_seq_bn"
pretrained_adapter_hub_name = f"pretrain_{pretrained_adapter_name}_adpater"

In [4]:
print(pretraining_adapter_config.__class__.__name__)
print(pretrained_adapter_name)

SeqBnConfig
tapt_seq_bn


In [5]:


# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=pretraining_adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")

Pretrained Adapter Name: tapt_seq_bn


In [6]:
pretrain_model.active_adapters

Stack[tapt_seq_bn]

In [7]:
summary = pretrain_model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
tapt_seq_bn              bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [10]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [ ]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir=pretrained_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=11,
    weight_decay=0.01,
    # Logging and eval during training settings
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=100,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
)

In [ ]:
before_pretrain_eval_results = pretrain_trainer.evaluate()
before_pretrain_eval_results_loss = before_pretrain_eval_results["eval_loss"]
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(before_pretrain_eval_results )

In [ ]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

In [ ]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

In [ ]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name
)

In [ ]:
import gc

print_gpu_utilization()

del pretraining_data_collator
del pretrain_train_dataset
del pretrain_eval_dataset
del pretrain_model
del pretrain_trainer
gc.collect()

print_gpu_utilization()

# Classification - Shared Setup

In [ ]:
# THIS MUST BE SET TO MATCH  classification_adapter_type
from adapters import SeqBnConfig

classification_adapter_config = SeqBnConfig()

In [ ]:
print(classification_adapter_config.__class__.__name__)
print(classification_adapter_type)

In [ ]:
from transformers import DataCollatorWithPadding

classification_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Classification

In [ ]:
from datasets import load_dataset

classification_train_dataset, classification_eval_dataset= load_dataset(classification_dataset_name, split=[classification_train_split,classification_eval_split])

print(classification_train_dataset)
print(classification_eval_dataset)

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_no_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
# Add a new adapter
classficiation_no_pretrain_model.add_adapter(classification_adapter_name, config=classification_adapter_config)

# Add head for classification modeling
classficiation_no_pretrain_model.add_classification_head(
    classification_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_no_pretrain_model.train_adapter(classification_adapter_name)

print(f"Classification Adapter Name: {classification_adapter_name}")

In [ ]:
classficiation_no_pretrain_model.active_adapters

In [ ]:
summary = classficiation_no_pretrain_model.adapter_summary()
print(summary)

In [ ]:
from transformers import TrainingArguments

classification_training_args = TrainingArguments(
    output_dir=classification_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Logging and eval during training settings
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=1000,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
from adapters import AdapterTrainer

classification_trainer = AdapterTrainer(
    model=classficiation_no_pretrain_model,
    args=classification_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
classification_training_results = classification_trainer.train()
print(classification_training_results)

In [ ]:
classification_eval_results = classification_trainer.evaluate()
classification_eval_loss = classification_eval_results["eval_loss"]
classification_eval_f1 = classification_eval_results["eval_f1_macro"]
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.2f}")
print(classification_eval_results)

In [ ]:
classficiation_no_pretrain_model.push_adapter_to_hub(
    classification_adapter_hub_name,
    classification_adapter_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_hub_name}")

In [ ]:
import gc

print_gpu_utilization()

del classficiation_no_pretrain_model
del classification_trainer
gc.collect()

print_gpu_utilization()

# Classification from Pre-training

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_from_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
# Load pre-trained adapter
load_name = f"BigTMiami/{pretrained_adapter_hub_name}"

# Load Pretrained adapter without head
loaded_adapter_name = classficiation_from_pretrain_model.load_adapter(load_name, with_head=False, set_active=True)

print(f"Loaded Pretrain Adapter Name: {load_name}")

In [ ]:
summary = classficiation_from_pretrain_model.adapter_summary()
print(summary)

In [ ]:
# Add a new adapter
classficiation_from_pretrain_model.add_adapter(classification_adapter_after_pretrained_name, config=classification_adapter_config)

# Add head for classification modeling
classficiation_from_pretrain_model.add_classification_head(
    classification_adapter_after_pretrained_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_from_pretrain_model.train_adapter(classification_adapter_after_pretrained_name)

print(f"Classification after Pretrain Adapter Name: {classification_adapter_after_pretrained_name}")

In [ ]:
summary = classficiation_from_pretrain_model.adapter_summary()
print(summary)

In [ ]:
#Set pretrained active, but don't include it in the train command
classficiation_from_pretrain_model.set_active_adapters([loaded_adapter_name, classification_adapter_after_pretrained_name])

In [ ]:
summary = classficiation_from_pretrain_model.adapter_summary()
print(summary)

In [ ]:
from transformers import TrainingArguments

classification_from_pretrained_training_args = TrainingArguments(
    output_dir=classification_adapter_after_pretrained_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Logging and eval during training settings
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=1000,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
from adapters import AdapterTrainer

classification_from_pretrained_trainer = AdapterTrainer(
    model=classficiation_from_pretrain_model,
    args=classification_from_pretrained_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)
print_gpu_utilization()

In [ ]:
classification_from_pretrained_training_results = classification_from_pretrained_trainer.train()
print(classification_from_pretrained_training_results)
print_gpu_utilization()

In [ ]:
classification_from_pretrained_eval_results = classification_from_pretrained_trainer.evaluate()
classification_from_pretrained_eval_loss = classification_from_pretrained_eval_results["eval_loss"]
classification_from_pretrained_eval_f1 = classification_from_pretrained_eval_results["eval_f1_macro"]
print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_f1:.2f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_loss:.2f}")
print(classification_from_pretrained_eval_results)
print_gpu_utilization()

In [ ]:
classficiation_from_pretrain_model.push_adapter_to_hub(
    classification_adapter_after_pretrained_hub_name,
    classification_adapter_after_pretrained_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_after_pretrained_hub_name}")

In [ ]:
print_gpu_utilization()

In [ ]:
end_time = time()
total_time = end_time - start_time

In [ ]:
print("Summary")
print("----------------")
print(f"Pretraining Adapter Type:{pretraining_adapter_type}")
print(f"Pretrain Overall Train Percentage:{pretrain_percentage_final}")
print("")
print(f"Classification Adapter Type:{classification_adapter_type}")
print(f"Classification Train Percentage:{classfication_percentage_final}")
print("")
print("Running Time")
print("----------------")
print(f"Total Time: {total_time:.0f} seconds OR {total_time / 60.0 :.0f} minutes OR {total_time / 3600.0 :.1f} hours ")
print("")
print("Pre-training Loss Change - should be high because adapter is completely random")
print("----------------")
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print("")
print("Classification Loss Change - should not be that different because both are starting from a blank classifier")
print("----------------")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.4f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_loss:.4f}")
print("")
print("Classification F1 Change - hopefully we get a few points improvement here!")
print("----------------")
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_f1:.2f}")

print("")
pretraining_f1_improvement = (100.0 * (  classification_from_pretrained_eval_f1 - classification_eval_f1))
print(f"Pretraining F1 Improvement: {pretraining_f1_improvement:.2f}")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()